In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob
import os
import tensorflow as tf
from tensorflow import keras
print(tf.version.VERSION)

2023-01-22 03:08:12.385876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd /content/drive/My Drive/Colab Notebooks/Dissertation

In [4]:
# !pip install pyyaml h5py  # Required to save models in HDF5 format

In [5]:
def get_house_path(house):
    return f'data/ukdale-parsed-chunks/house_{house}/'
    
def get_chunk_path(house, chunk):
    return get_house_path(house) + f'chunk_{chunk}.dat'

def get_num_chunks(house):
    return len(glob.glob(get_house_path(house) + 'chunk_*[0-9].dat'))

def read_file(house, chunk, labels):
    file = get_chunk_path(house, chunk)
    print(f'reading file {file}; for house {house} and chunk {chunk}');
    
    dtypes = {}
    for label in labels[house]:
        dtypes[label] = 'float32'
    df = pd.read_table(file, sep = '\t', header=0, names = labels[house], 
                                       dtype = dtypes) 

    return df

In [6]:
def parse_data(df):
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    df.set_index(df['timestamp'].values, inplace=True)
    df.drop(['unix_time'], axis=1, inplace=True)

    df['timeslice'] = df.timestamp.dt.hour

    return df

In [7]:
def read_labels():
    labels = {}
    for house in range(1, 2):
        fileName = get_chunk_path(house, 1)
        house_labels = pd.read_csv(fileName, sep = '\t', nrows=1).columns.tolist()
        labels[house] = house_labels
    return labels

In [8]:
def get_house_data_generator(house):
    labels = read_labels()
    num_chunks = get_num_chunks(house)
    for i in range(1, num_chunks + 1):
        if int(i) == 1:
            print(f'reading house {house}; chunk 1');

        df = read_file(house, i, labels)
        df = parse_data(df)
        
        print(f'read house {house}; chunk {i}; df.shape is {df.shape}')
    
        yield df

In [9]:
def get_merged_chunks(house, num_chunks):
    max_num_chunks = get_num_chunks(house)
    num_chunks = max_num_chunks if num_chunks > max_num_chunks else num_chunks
    house_gen = get_house_data_generator(house)
    data = [next(house_gen) for i in range(num_chunks)]

    return pd.concat(data)


In [10]:
def get_all_data_generators():
    gen = {}
    for house in range(1,2):
        gen[house] = get_house_data_generator(house)
        
    return gen

In [11]:
labels = read_labels()
for house in range(1,2):
    print('House {}: '.format(house), labels[house], '\n')

House 1:  ['mains_active', 'mains_apparent', 'mains_rms', 'aggregate_apparent', 'ft_boiler', 'ft_solar_thermal_pump', 'ft_washing_machine', 'ft_dishwasher', 'ft_tv', 'ft_kitchen_lights', 'ft_htpc', 'ft_kettle', 'ft_toaster', 'ft_fridge', 'ft_microwave', 'ft_amp_livingroom', 'ft_adsl_router', 'ft_livingroom_s_lamp', 'ft_lighting_circuit', 'ft_subwoofer_livingroom', 'ft_livingroom_lamp_tv', 'ft_kitchen_phone&stereo', 'ft_coffee_machine', 'ft_gas_oven', 'ft_data_logger_pc', 'ft_office_lamp2', 'aggregate_active', 'unix_time'] 



In [12]:
# gen = get_all_data_generators()

In [13]:
# house_gen = get_house_data_generator(1)

In [14]:
def print_heads_tails(df, h=True,t=True):
    print(f'House {house}, data has shape: {df.shape}')
    if h:
        display(df.head(2))

    print()

    if t:
        display(df.tail(2))

In [ ]:
ref_chunk_df = get_merged_chunks(1, 2)
print_heads_tails(ref_chunk_df)

reading house 1; chunk 1
reading file data/ukdale-parsed-chunks/house_1/chunk_1.dat; for house 1 and chunk 1


In [ ]:
def get_dates(house_df, house):
    dates = [str(time)[:10] for time in house_df.index.values]
    dates = sorted(list(set(dates)))
    print('House {0} data contain {1} days from {2} to {3}.'.format(house,len(dates),dates[0], dates[-1]))
    print(dates, '\n')

    return dates

In [ ]:
dates = {}
# for house in range(1,2):
dates[1] = get_dates(ref_chunk_df, 1)

In [ ]:
# Use day 2, house 1, to get number of df rows per day
rows_per_day = ref_chunk_df.loc[:dates[1][2]].shape[0]
feature = 'ft_kettle'

In [ ]:
# Plot first 10 days of device
def plot_ft_days(df, label, n_days):
    days_series = df.loc[:dates[house][n_days]][label]
    print(days_series.shape)
    days_series_bins = days_series.values.reshape(-1, n_days)
    print(days_series_bins.shape)
    fig, axes = plt.subplots((n_days+1)//2,2, figsize=(24, n_days*2) )
    for i in range(days_series_bins.shape[-1]):
        series = days_series_bins[:,i]
        axes.flat[i].plot(series, alpha = 0.6)
        axes.flat[i].set_title(f'day {i+1}', fontsize = '15')
    plt.suptitle(f'First n_days for {label}', fontsize = '30')
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)

In [ ]:
plot_ft_days(ref_chunk_df, feature, 10)

In [ ]:
# Separate house 1 data into train, validation and test data
n_days = len(dates[house])
train_index = math.ceil(n_days *0.5)
test_index = math.ceil(n_days *0.8)
df1_train = ref_chunk_df.loc[:dates[1][train_index]]
df1_val = ref_chunk_df.loc[dates[1][train_index]:dates[1][test_index]]
df1_test = ref_chunk_df.loc[dates[1][test_index]:]
print('df_train.shape: ', df1_train.shape)
print('df_val.shape: ', df1_val.shape)
print('df_test.shape: ', df1_test.shape)

In [ ]:
# Global vars
ref_max = df1_train[['mains_active']].values.max()
seq_per_batch = 1000
seq_len = 128
# seq_per_batch = 64
# seq_len = 128

In [ ]:
def reshape_x(x):
    # return np.reshape(x, (x.shape[0], n_bins, 1))
    return np.reshape(x, (x.shape[0], 1, 1))

class KerasBatchGenerator(object):
    # we default stride to 1 since we dont want future values to affect past values
    def __init__(self, mains, meter, seq_per_batch, seq_len, stride=1):
        assert(len(mains) == len(meter))
        self.data_x = mains
        self.data_y = meter
        self.seq_per_batch = seq_per_batch # the number of seqeunces per batch
        self.seq_len = seq_len # the length of each sequence
        self.stride = stride
        self.current_idx = 0
        self.data_len = len(self.data_x)

    def load_sequence(self, x, y, seq_index):
        seq = self.data_x[self.current_idx:self.current_idx + self.seq_len]
        # Pad initial sequences since they will surely have zero padding if seq_len > 
        x[seq_index, self.seq_len - len(seq):] = seq
        y[seq_index] = self.data_y[self.current_idx + 1]

    def generate(self):
        x = np.zeros((self.seq_per_batch, self.seq_len, 1), dtype=np.float32)
        y = np.zeros((self.seq_per_batch), dtype=np.float32)
        while True:
            for seq_index in range(self.seq_per_batch):
                if self.current_idx + self.seq_len >= self.data_len:
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                self.load_sequence(x, y, seq_index)
                self.current_idx += self.stride
            yield x, y

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, LSTM, Bidirectional, Flatten, Dropout, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from timeit import default_timer as timer

def create_model():
    # Rework of:
    # https://github.com/JackKelly/neuralnilm_prototype/blob/2119292e7d5c8a137797ad3c9abf9f37e7f749af/scripts/e567.py
    # https://github.com/OdysseasKr/neural-disaggregator/blob/master/DAE/daedisaggregator.py
    model = Sequential()

    # 1D Conv
    model.add(Conv1D(8, 4, activation="linear", padding="same", strides=1))
#     model.add(Conv1D(8, 4, activation="linear", input_shape=(seq_len, 1), padding="same", strides=1))
    model.add(Flatten())

    # Fully Connected Layers
    model.add(Dropout(0.2))
    model.add(Dense((seq_len-0)*8, activation='relu'))

    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))

    model.add(Dropout(0.2))
    model.add(Dense((seq_len-0)*8, activation='relu'))

    model.add(Dropout(0.2))

    # 1D Conv
    model.add(Reshape(((seq_len-0), 8)))
    model.add(Conv1D(1, 4, activation="linear", padding="same", strides=1))

    adam_opt = Adam(lr = 1e-1)
#     sgd_opt = SGD(lr = 1e-1, nesterov=True)
    model.compile(loss='mse', optimizer=adam_opt,metrics=['accuracy', RootMeanSquaredError(), MeanAbsoluteError()])

    return model

In [ ]:
model = create_model()
# model.summary()

In [ ]:
checkpoint_path = f"dae_training_{feature}/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def train(model, feature, epochs = 1):
    x_train = df1_train[['mains_active']].values
    x_val = df1_val[['mains_active']].values

    y_train = df1_train[[feature]].values
    y_val = df1_val[[feature]].values

    y_train_norm = y_train / ref_max
    y_val_norm = y_val / ref_max
    x_train_norm = x_train / ref_max
    x_val_norm = x_val / ref_max

    print(x_train_norm.shape, y_train_norm.shape, x_val_norm.shape, y_val_norm.shape)

    extra_train = seq_len - (len(x_train) % seq_len)

    x_train_norm = np.append(x_train_norm, np.zeros(extra_train)).reshape((-1, 1))
    y_train_norm = np.append(y_train_norm, np.zeros(extra_train)).reshape((-1, 1))

    extra_val = seq_len - (len(x_val) % seq_len)
    x_val_norm = np.append(x_val_norm,  np.zeros(extra_val)).reshape((-1, 1))
    y_val_norm = np.append(y_val_norm,  np.zeros(extra_val)).reshape((-1, 1))
    
    print(x_train_norm.shape, y_train_norm.shape, x_val_norm.shape, y_val_norm.shape)

    x_train_reshaped = np.reshape(x_train_norm, (len(x_train_norm) // seq_len, seq_len, 1))
    y_train_reshaped = np.reshape(y_train_norm, (len(y_train_norm) // seq_len, seq_len, 1))
    x_val_reshaped = np.reshape(x_val_norm, (len(x_val_norm) // seq_len, seq_len, 1))
    y_val_reshaped = np.reshape(y_val_norm, (len(y_val_norm) // seq_len, seq_len, 1))

    print(f'sequence_len {seq_len}')
    print(f'epochs {epochs}')
    print(f'Input initial shape {x_train.shape}; input final shape {x_train_reshaped.shape}')
    print(f'Output initial shape {y_train.shape}; Output final shape {y_train_reshaped.shape}')

    time_cb = TimingCallback()
    cp_cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq=500,
                                                 verbose=2)
    callbacks = [time_cb, cp_cb]
    
        
    n_train_batches = len(x_train_norm) // seq_len
    n_val_batches = len(x_val_norm) // seq_len
    train_data_generator = KerasBatchGenerator(x_train_norm, y_train_norm, seq_per_batch=n_train_batches, seq_len=seq_len)
    val_data_generator = KerasBatchGenerator(x_val_norm, y_val_norm, seq_per_batch=n_val_batches, seq_len=seq_len)
    
#     history = model.fit(train_data_generator.generate(), steps_per_epoch=n_train_batches,
#                     epochs=epochs,
#                     validation_data=val_data_generator.generate(), callbacks=callbacks, workers=2)

    history = model.fit(x_train_reshaped, y_train_reshaped, epochs=epochs, batch_size=seq_per_batch, validation_data=(x_val_reshaped, y_val_reshaped), callbacks=callbacks, verbose=2)
    # history = model.fit(x_train_reshaped, y_train_reshaped, epochs=epochs, batch_size=seq_per_batch, validation_data=(x_val_reshaped, y_val_reshaped), callbacks=callbacks, shuffle=True)

    return model, history, sum(time_cb.logs)

def view_model_results(model, history):
    fig = plt.figure()
    fig.add_subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='test loss')
    plt.legend()
    plt.grid(True)
    plt.xlabel('epoch')

    # fig.add_subplot(1,2,2)
    # plt.plot(history.history['accuracy'], label='train accuracy')
    # plt.plot(history.history['val_accuracy'], label='test accuracy')
    # plt.legend()
    # plt.grid(True)
    # plt.xlim([0,3])
    # plt.ylim([0,1.0])
    # plt.xlabel('epoch')

    print(history.history.keys())

In [ ]:
# Load model
LOAD_MODEL=False

In [ ]:
if LOAD_MODEL:
  # Loads the weights
  model.load_weights(checkpoint_path)
else:
  model, history, time_spent = train(model, feature, epochs = 1000)
  print('Time spent', time_spent)
  view_model_results(model, history)

In [ ]:
def predict(model, x_test):
    pred = model.predict(x_test)
    
    return pred

def plot_prediction_windows(label, y_test, y_pred, use_active = False, n_samples = seq_len):
    y_test_mean = y_test.mean()
    n_plots_max = 16
    n_rows = n_samples
    n_plots = len(y_test) // n_rows
    y_test_bins = np.resize(y_test, (n_rows, n_plots))
    y_pred_bins = np.resize(y_pred, (n_rows, n_plots))
    fig, axes = plt.subplots((n_plots_max+1)//2,2, figsize=(24, n_plots_max*2) )
    plot_counter = 0
    for i in range(y_test_bins.shape[-1]):
        if plot_counter < n_plots_max:
            y_test_series = y_test_bins[:,i]
            y_pred_series = y_pred_bins[:,i]
            if use_active: 
                window_mean = y_test_series.mean()
                if y_test_series.mean() >= y_test_mean * 0.5:
                    axes.flat[plot_counter].plot(y_test_series, color = 'blue', alpha = 0.6, label = 'True value')
                    axes.flat[plot_counter].plot(y_pred_series, color = 'red', alpha = 0.6, label = 'Predicted value')
                    axes.flat[plot_counter].set_title(f'window {plot_counter+1}; mean {window_mean}', fontsize = '15')
                    plot_counter += 1
            else: 
                axes.flat[plot_counter].plot(y_test_series, color = 'blue', alpha = 0.6, label = 'True value')
                axes.flat[plot_counter].plot(y_pred_series, color = 'red', alpha = 0.6, label = 'Predicted value')
                axes.flat[plot_counter].set_title(f'window {plot_counter+1}', fontsize = '15')
                plot_counter += 1
    plt.suptitle(f'Sample n_window predictions for {label}; data mean {y_test_mean} ', fontsize = '30')
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)

In [ ]:
x_test = df1_test[['mains_active']].values
y_test = df1_test[[feature]].values
original_len = len(x_test)

x_test_norm = x_test / ref_max

print(x_test.shape)

extra_test = seq_len - (len(x_test) % seq_len)

x_test_norm = np.append(x_test_norm, np.zeros(extra_test))
x_test_reshaped = np.reshape(x_test_norm, (len(x_test_norm) // seq_len, seq_len, 1))

In [ ]:
# Predict from test split
y_pred = predict(model, x_test_reshaped)
y_pred_reshaped = np.reshape(y_pred, (original_len + extra_test))[:original_len]

In [ ]:
# Plot predictions
print(y_test.shape)
print(y_pred_reshaped.shape)

y_pred_reshaped[y_pred_reshaped < 0] = 0

print(y_test.mean())
print(y_pred_reshaped.mean())

y_pred_denorm = y_pred_reshaped * ref_max

print(y_test.mean())
print(y_pred_denorm.mean())

In [ ]:
n_samples = int(rows_per_day)
plot_prediction_windows(feature, y_test, y_pred_denorm, use_active=False, n_samples=n_samples)

In [ ]:
n_samples = int(rows_per_day * 0.2)
plot_prediction_windows(feature, y_test, y_pred_denorm, use_active=False, n_samples=n_samples)

In [ ]:
n_samples = 80
plot_prediction_windows(feature, y_test, y_pred_denorm, use_active=True, n_samples=n_samples)

In [ ]:
n_samples = 600
plot_prediction_windows(feature, y_test, y_pred_denorm, use_active=True, n_samples=n_samples)

In [ ]:
# train_and_present_results('ft_kettle')

In [ ]:
# train_and_present_results('ft_fridge')

In [ ]:
# train_and_present_results('ft_washing_machine')

In [ ]:
# train_and_present_results('ft_microwave')

In [ ]:
# train_and_present_results('ft_dishwasher')